In [50]:
# Import libraries
import numpy as np
import pandas as pd

In [51]:
# Read in nonvoters dataset
non_voters = pd.read_csv('nonvoters_data.csv')
sample_size = len(non_voters)

# Change pandas settings to not truncate output
pd.set_option('display.max_columns', None)

In [52]:
# Print data types an non-null count for each feature  
#non_voters.info(verbose=True, show_counts=True)

# Change categorical features to be of type (this is all columns except RespId, weight, and ppage)
non_categorical = ['RespId', 'weight', 'ppage']
for column in non_voters.columns:
    if column not in non_categorical:
        non_voters[column] = non_voters[column].astype('category')

#print(non_voters['gender'].cat.codes)
#print(non_voters['gender'].cat.categories)
#print(non_voters['gender'].head())

# Print descriptive statistics for dataset
#print(non_voters.describe(exclude=['int64', 'float64']))

print(non_voters['income_cat'].value_counts())
print(non_voters['income_cat'].cat.categories)
print(non_voters['educ'].value_counts())
print(non_voters['educ'].cat.categories)

$75-125k          1628
Less than $40k    1418
$40-75k           1396
$125k or more     1394
Name: income_cat, dtype: int64
Index(['$125k or more', '$40-75k', '$75-125k', 'Less than $40k'], dtype='object')
College                2330
High school or less    1796
Some college           1710
Name: educ, dtype: int64
Index(['College', 'High school or less', 'Some college'], dtype='object')


In [54]:
# Check for NA values
na_feature_count = 0
for column in non_voters.columns:
    na_values = non_voters[column].isna().sum()
    if(na_values > 0):
        print(f'Column {column} has {na_values} NA values.')
        non_voters.drop(column, axis=1, inplace=True)
        na_feature_count += 1

print(f'{na_feature_count} features have NA values.')

# Check for -1 values
negone_feature_count = 0
for column in non_voters.columns:
    if(-1 in non_voters[column].value_counts()):
        negone_values = non_voters[column].value_counts()[-1]
        print(f'Column {column} has {negone_values} -1 values.')
        negone_feature_count += 1

print(f'{negone_feature_count} features have -1 values.')

0 features have NA values.
Column Q2_1 has 35 -1 values.
Column Q2_2 has 36 -1 values.
Column Q2_3 has 36 -1 values.
Column Q2_4 has 34 -1 values.
Column Q2_5 has 28 -1 values.
Column Q2_6 has 31 -1 values.
Column Q2_7 has 44 -1 values.
Column Q2_8 has 26 -1 values.
Column Q2_9 has 36 -1 values.
Column Q2_10 has 38 -1 values.
Column Q3_1 has 39 -1 values.
Column Q3_2 has 53 -1 values.
Column Q3_3 has 54 -1 values.
Column Q3_4 has 32 -1 values.
Column Q3_5 has 45 -1 values.
Column Q3_6 has 34 -1 values.
Column Q4_1 has 39 -1 values.
Column Q4_2 has 39 -1 values.
Column Q4_3 has 33 -1 values.
Column Q4_4 has 38 -1 values.
Column Q4_5 has 44 -1 values.
Column Q4_6 has 33 -1 values.
Column Q5 has 31 -1 values.
Column Q6 has 47 -1 values.
Column Q7 has 61 -1 values.
Column Q8_1 has 47 -1 values.
Column Q8_2 has 49 -1 values.
Column Q8_3 has 42 -1 values.
Column Q8_4 has 34 -1 values.
Column Q8_5 has 49 -1 values.
Column Q8_6 has 26 -1 values.
Column Q8_7 has 42 -1 values.
Column Q8_8 has 29